In [1]:
import openai, json, pandas as pd, os


In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
def query(prompt, **kwargs):
  """
  wrapper for the API to save the prompt and the result
  """
  props = {
  "engine":"text-davinci-003",
  "temperature":0,
  "max_tokens":250,
  "stop":"\n\n",
  **kwargs
  }

  r = openai.Completion.create(prompt=prompt, **props)["choices"][0]["text"].strip()
  return r
     


Making sure this works

In [4]:
query("q: what is 1+1?\na:", stop="\n")


'2'

Let's go and start creating some frew shots from the 

In [5]:
train = pd.read_json("anli_v1.0/R3/train.jsonl",lines=True)


In [6]:
train.label.value_counts()

n    40778
e    32292
c    27389
Name: label, dtype: int64

In [7]:
basePrompt = """You are an expert analyst evaluating where claims are contradictions, entailments, or neutral with respect to a background.

{
"StoryA": "The others gave in very soon , and longed to be friends , for now there was no Daisy to pet and cook for them ; no Nan to amuse and doctor them ; and , worst of all , no Mrs. Jo to make home life pleasant and life easy for them .<br>To their great affliction , Mrs. Jo seemed to consider herself one of the offended girls , for she hardly spoke to the outcasts , looked as if she did not see them when she passed , and was always too busy now to attend to their requests .",
"StoryB": "Nan was there to doctor them.",
"Point of Contention": "Nan being a doctor",
"Story A Relevance": "No nan to amuse and doctor them",
"Story B Relevance": "Nan was there to doctor them",
"Explaining the Differences for a Child": "In the first story, 'No Nan to amuse and doctor them' means that there was no one to take care of them. In the second story, 'Nan was there to doctor them' means that Nan was there to take care of them. The first sentence says the opposite of the second sentence.",
"Verdict": "Contradiction"
},
{
"StoryA": "I wonder how he expects us to put forward a strong position on these issues if we are not part of the World Trade Organization or if we were not in a position, with meetings like Seattle, where we can pull together different countries from around the world that have a similar position to ours.",
"StoryB": "there were never meetings in Seattle",
"Point of Contention": "Meetings in Seattle",
"Story A Relevance": "with meetings like Seattle, where we can pull together different countries from around the world that have a similar position to ours.",
"Story B Relevance": "there were never meetings in Seattle",
"Explaining the Differences for a Child": "In the first story, it says that there were meetings in Seattle where different countries could come together. In the second story, it says that there were never meetings in Seattle. The first sentence suggests that there were meetings in Seattle, while the second sentence suggests that there were never any meetings in Seattle.",
"Verdict": "Contradiction"
},
{
"StoryA": "Luciano Chailly (born Ferrara, January 19, 1920 – died Milan, December 24, 2002) was an Italian composer and arts administrator of French descent. He was the father of harpist Cecilia Chailly, conductor Riccardo Chailly and journalist Floriana Chailly. As a composer, Chailly was best known for his operas, many of which were composed to libretti by Dino Buzzati.",
"StoryB": "Luciano Chailly was born in January",
"Point of Contention": "Luciano Chailly's birth month",
"Story A Relevance": "Luciano Chailly (born Ferrara, January 19, 1920",
"Story B Relevance": "Luciano Chailly was born in January",
"Explaining the Differences for a Child": "In the first story, it says that Luciano Chailly was born in January on the 19th. In the second story, it says that Luciano Chailly was born in January. The first sentence is more specific, saying that Luciano Chailly was born on the 19th of January, while the second sentence is more general, saying that Luciano Chailly was born in January.",
"Verdict": "Entailment"
},
{
"StoryA": "Segura had X-rays on his hand after getting hit by a pitch Friday against the Astros, Shannon Drayer of 710 ESPN Seattle reports. Segura was hit in the eighth inning but did not leave the game. He appears to have avoided a serious injury but is considered day-to-day.",
"StoryB": "Segura was hit Friday.,
"Point of Contention": "Segura being hit by a pitch",
"Story A Relevance": "Segura had X-rays on his hand after getting hit by a pitch Friday against the Astros",
"Story B Relevance": "Segura was hit Friday.",
"Explaining the Differences for a Child": "In the first story, it says that Segura had X-rays on his hand after getting hit by a pitch on Friday. In the second story, it says that Segura was hit on Friday. The first sentence is more specific, saying that Segura was hit by a pitch, while the second sentence is more general, saying that Segura was hit on Friday.",
"Verdict": "Entailment"
},
{
"StoryA": "Grudge<br>Tom had a grudge against Ana. He decided he was going to slit her throat. Tom waited in the shadows by Ana's house. When he saw her red coat he lunged out, slicing and stabbing! Tom had only killed Ana's friend, who had borrowed her coat.",
"StoryB": "Ana lost her coast",
"Point of Contention": "Ana losing her coat",
"Story A Relevance": "When he saw her red coat he lunged out, slicing and stabbing!",
"Story B Relevance": "Ana lost her coast",
"Explaining the Differences for a Child": "In the first story, it says that Tom lunged out when he saw Ana's red coat. In the second story, it says that Ana lost her coat. The first sentence suggests that Ana still had her coat, while the second sentence suggests that Ana no longer had her coat.",
"Verdict": "Contradiction"
},
{
"StoryA": "London Calling is the third studio album by English punk rock band the Clash. It was released as a double album in the United Kingdom on 14 December 1979 by Columbia Records, and in the United States in January 1980 by Epic Records. "London Calling" is an album that incorporates a range of styles, including punk, reggae, rockabilly, ska, New Orleans R&B, pop, lounge jazz, and hard rock.",
"StoryB": "Thanks to the rockabilly and R&B the album was released in the United States in 1980.",
"Point of Contention": "Album being released in the United States in 1980",
"Story A Relevance": "London Calling was released in the United States in January 1980 by Epic Records.",
"Story B Relevance": "Thanks to the rockabilly and R&B the album was released in the United States in 1980.",
"Explaining the Differences for a Child": "In the first story, it says that the album was released in the United States in January 1980. In the second story, it says that the album was released in the United States in 1980 thanks to the rockabilly and R&B the album. They both agree that it was released in 1980 but there is nothing in the first story that relates to whether it was released thanks to the rockabilly and R&B the album.",
"Verdict": "Neutral"
}

"""

In [8]:
def createTargetExample(row):
  return f"""{{
    "StoryA": "{row.context}",
    "StoryB": "{row.hypothesis}",
    "Point of Contention":"""
    
def evaluateAnswer(res):
    if "entailment" in res.lower()[-30:]:
        return 'e'
    elif "contradiction" in res.lower()[-30:]:
        return 'c'
    return 'n'

In [9]:
dev = pd.read_json("anli_v1.0/R3/dev.jsonl",lines=True)

In [10]:
dev.head()

,uid,context,hypothesis,label,model_label,emturk,genre,reason,tag
0,50a5d4bc-bb8b-44f9-ba07-95dfaf5536ab,one of the orders issued by Ochola in April Lo...,The decision to move the photocopier business ...,e,n,False,news,I made an obvious inference from the text that...,r3_dev
1,08789750-b5fe-478d-8d3f-06a01e3a8b1e,"If you can dream it, you can achieve it — unle...",The crowd believed they knew the name of the g...,e,n,False,news,"Because the crowd was chanting its name, the c...",r3_dev
2,a960c8cc-2f55-4269-a028-7f714479b068,"The Kremlin described as ""complete nonsense"" o...",The Kremlin used the word nonsense multiple ti...,e,n,False,news,I based my statement on supposed word count ba...,r3_dev
3,95e97907-0267-45ae-89fc-7e1fce39ee77,Police said that a 21-year-old man was discove...,The victim was less than a quarter century old,e,c,False,news,21 is less than 25. I think the system got it ...,r3_dev
4,5bd3435a-e52e-4688-9bc3-746b47c53469,Hurricane Harvey has caused devastating floods...,Houston is not the only area impacted by Hurri...,e,n,False,news,Louisiana was also impacted,r3_dev


In [11]:
results_dict = {}
correct = 0
done = 0

In [14]:

for row in dev.iterrows():
    if row[0] in results_dict:
        continue
    newPrompt = basePrompt + createTargetExample(row[1])
    res = query(newPrompt, stop="}")
    results_dict[row[0]] = {
        "res": res,
        "label": row[1].label,
        "pred": evaluateAnswer(res)
    }
    done += 1
    if evaluateAnswer(res) == row[1].label:
        correct += 1
    print(f"Done {done} of {len(dev)}. Score: {correct}/{done} ({correct/done*100:.2f}%)")
    if done % 10 == 0:
        with open("textdavinci003-inDepthFewShot.json", "w") as f:
            json.dump(results_dict, f)    


Done 625 of 1200. Score: 314/625 (50.24%)
Done 626 of 1200. Score: 315/626 (50.32%)
Done 627 of 1200. Score: 315/627 (50.24%)
Done 628 of 1200. Score: 315/628 (50.16%)
Done 629 of 1200. Score: 315/629 (50.08%)
Done 630 of 1200. Score: 315/630 (50.00%)
Done 631 of 1200. Score: 315/631 (49.92%)
Done 632 of 1200. Score: 315/632 (49.84%)
Done 633 of 1200. Score: 315/633 (49.76%)
Done 634 of 1200. Score: 315/634 (49.68%)
Done 635 of 1200. Score: 315/635 (49.61%)
Done 636 of 1200. Score: 316/636 (49.69%)
Done 637 of 1200. Score: 316/637 (49.61%)
Done 638 of 1200. Score: 317/638 (49.69%)
Done 639 of 1200. Score: 318/639 (49.77%)
Done 640 of 1200. Score: 319/640 (49.84%)
Done 641 of 1200. Score: 319/641 (49.77%)
Done 642 of 1200. Score: 319/642 (49.69%)
Done 643 of 1200. Score: 319/643 (49.61%)
Done 644 of 1200. Score: 319/644 (49.53%)
Done 645 of 1200. Score: 319/645 (49.46%)
Done 646 of 1200. Score: 319/646 (49.38%)
Done 647 of 1200. Score: 319/647 (49.30%)
Done 648 of 1200. Score: 319/648 (

In [ ]:
with open("textdavinci003-inDepthFewShot.json", "w") as f:
    json.dump(results_dict, f)    
